In [ ]:
from pathlib import Path
from typing import List, Tuple, Dict

import torch 
import numpy as np 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import scipy.io
import visu3d as v3d

In [ ]:
H, W = (875, 1920)
outs = torch.load("checkpoints/dust3r_out.pth")

pts_final = []
v3dcams: List[v3d.Camera] = []
for i in range(len(outs["poses"])):
  f = outs["focals"][i, 0]
  cam_spec = v3d.PinholeCamera.from_focal(
      resolution=(H, W),
      focal_in_px=f,
  )
  pose = outs["poses"][i]
  R, t = pose[:3,:3], pose[:3,3:].T
  pts = outs["pts3d"][i]
  # pts = outs["pts3d"][i]@R.T + t 
  pts_final.append(pts)
  T = pose
  v3dcam = v3d.Camera(
      spec=cam_spec,
      world_from_cam=v3d.Transform.from_matrix(T)
  )
  v3dcams.append(v3dcam)

pts_final = torch.stack(pts_final).view(-1,3).numpy()

x = pts_final[::10,0]
y = pts_final[::10,1]
z = pts_final[::10,2]

In [ ]:
pts_final.shape

In [ ]:
# Create 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=x, y=y, z=z,
    mode="markers",
    marker=dict(
        size=1,
        color="blue",  # Set color to blue
        opacity=0.8
    )
)])

# Update layout
fig.update_layout(scene=dict(
                    xaxis_title="X AXIS",
                    yaxis_title="Y AXIS",
                    zaxis_title="Z AXIS"),
                width=700,
                margin=dict(r=20, b=10, l=10, t=10))

# Show the plot
fig.show()

In [ ]:
outs

In [ ]:
pose

In [ ]:
ptcloud = v3d.Point3d(p=pts_final[::10, :])
v3d.make_fig([ptcloud])

In [ ]:
x, y, z = ptcloud.p[:, 0], ptcloud.p[:, 1], ptcloud.p[:, 2]
matdata = {
    "scenex": x,
    "sceney": y,
    "scenez": z,
    "isbarrel": 0
}
scipy.io.savemat(Path("../results/barrel2-reconstr") / "barrel2-dense-seg.mat", matdata)